In [114]:
from shutil import copyfile

# Collect a list of images

In [9]:
image_names = !ls /Users/samuilstoychev/Desktop/RAF-DB/basic/Image/aligned

In [14]:
# Assert there are 15339 unique images (in accordance with RAF-DB's README file)
assert len(image_names) == 15339

# Collect ground truth and metadata

In [20]:
RAF_DB_ROOT_PATH = "/Users/samuilstoychev/Desktop/RAF-DB/basic"
RAF_DB_EMOTIONS_FILE = RAF_DB_ROOT_PATH + "/EmoLabel/list_patition_label.txt"
RAF_DB_ANNOTATIONS_PATH = RAF_DB_ROOT_PATH + "/Annotation/manual"

## Create emotion mapping

```
1: Surprise
2: Fear
3: Disgust
4: Happiness
5: Sadness
6: Anger
7: Neutral
```

In [72]:
emotion_mapping = dict() 

In [73]:
with open(RAF_DB_EMOTIONS_FILE, "r") as f: 
    for line in f: 
        line = line.strip()
        image_name, emotion = line.split(" ")
        image_id = image_name.split("_")[1].split(".")[0]
        emotion_mapping[image_id] = int(emotion)

In [75]:
# Assert every image has a corresponding mapping
assert len(emotion_mapping) == len(image_names)

## Collect feature mapping

In [79]:
annotation_files = !ls {RAF_DB_ANNOTATIONS_PATH}

In [82]:
# Assert there is an annotation file for each image
assert len(annotation_files) == len(image_names)

In [91]:
feature_mapping = dict()

for annotation_file in annotation_files: 
    image_id = annotation_file.split("_")[1]
    with open(RAF_DB_ANNOTATIONS_PATH + "/" + annotation_file) as f: 
        gender, race, age = [int(x.strip()) for x in f.readlines()[-3:]]
        assert (gender >= 0 and gender <= 2)
        assert (race >= 0 and race <= 2)
        assert (age >= 0 and age <= 4)
        feature_mapping[image_id] = {"gender": gender, "race": race, "age": age}

In [92]:
assert len(feature_mapping) == len(image_names)

In [105]:
class RAFDBImage: 
    RAF_DB_ROOT_PATH = "/Users/samuilstoychev/Desktop/RAF-DB/basic"
    
    def __init__(self, name): 
        self.name = name
        self.category = name.split("_")[0]
        self.id = name.split("_")[1]
        self.emotion = emotion_mapping[self.id]
        self.gender = feature_mapping[self.id]["gender"]
        self.race = feature_mapping[self.id]["race"]
        self.age = feature_mapping[self.id]["age"]      
    
    def __str__(self): 
        return "Name: {}, Emotion: {}, Gender: {}, Race: {}, Age: {}".format(
            self.name, self.emotion, self.gender, self.race, self.age
        )
    
    def __repr__(self): 
        return self.__str__()
    
    def get_final_name(self): 
        return "{}_{}_{}_{}.jpg".format(self.id, self.gender, self.race, self.age)

# Move files in the right folders

In [109]:
images = [RAFDBImage(image_name) for image_name in image_names]

In [119]:
for image in images: 
    SRC = "/Users/samuilstoychev/Desktop/RAF-DB/basic/Image/aligned/" + image.name
    DEST = "/Users/samuilstoychev/RAF_DB_preprocessed/{}/class_{}/{}".format(
        image.category, image.emotion, image.get_final_name()
    )
    copyfile(SRC, DEST)

# Note: Annotations are as follows

```
Gender
0: male		1: female	2: unsure
----------------------------------------------------------------------
Race	
0: Caucasian		1: African-American		2: Asian
----------------------------------------------------------------------
Age (5 ranges)	
0: 0-3	 1: 4-19		2: 20-39		3: 40-69		4: 70+
----------------------------------------------------------------------
```